In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext wurlitzer

In [3]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level = logging.INFO)

In [4]:
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil

In [5]:
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from IPython.display import display

config_file = Path("~/git/IonBeam_bundle/IonBeam/config/").expanduser()
config, actions = parse_config(config_file,
                    environment  = "local",
                    sources = ["meteotracker"]
                    )

In [6]:
from ionbeam.metadata.db import init_db
import sqlalchemy
Base = sqlalchemy.orm.declarative_base()
force_init = False

# if force_init or Base.metadata.reflect(config.globals.sql_engine) is None:
#     init_db(config.globals)

In [7]:
from uuid import UUID

id2action = {a.id : a for a in actions}
sources = [a for a in actions if isinstance(a, Source)]
processors = [a for a in actions if not isinstance(a, Source)]

chains = [[s,] for s in sources]
for a in processors:
    if not isinstance(a.match, UUID): 
        chains.append([str(a.match), a,])
        continue
    for c in chains:
        matches = a.match.int == c[-1].id.int
        if matches: c.append(a)
            

for c in chains:
    print(" --> ".join(str(a) for a in c))

MeteoTrackerSource --> ExtractMetaData --> CanonicaliseColumns --> ComputeDateTime --> ComputeMARSIdentifier --> AddMeteotrackerMetadata --> SimpleODCEncoder
[Match(state = 'odc_encoded')] --> FDBWriter()


In [8]:
sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

print("Sources")
for i, a in enumerate(sources):
    display(a)

print("Aggregators")
for i, a in enumerate(aggregators):
    display(a)

print("Actions")
for i, a in enumerate(stateless_actions):
    display(a)

Sources


id,2c4224d7-391f-4d36-a24f-2e33cd365f70
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/meteotracker/actions.yaml
mappings,"[RawVariable(name='datetime', key='time', type='datetime', unit=None, discard=False, canonical_va..."
finish_after,None
cache_version,3
use_cache,True
source,meteotracker
maximum_request_size,6:00:00
minimum_request_size,0:05:00
max_time_downloading,0:00:10
wkt_bounding_polygon,None


Aggregators
Actions


id,f4c4cf16-a0c2-4b90-b5dd-2e371dcca4f0
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/meteotracker/actions.yaml
match,2c4224d7-391f-4d36-a24f-2e33cd365f70
copy_metadata_to_columns,{'author': 'author'}
name,value
source_action_id,f4c4cf16-a0c2-4b90-b5dd-2e371dcca4f0
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data


id,df5c641b-c50e-4e00-9589-3e13538450c5
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/meteotracker/actions.yaml
match,f4c4cf16-a0c2-4b90-b5dd-2e371dcca4f0
mappings,"[RawVariable(name='datetime', key='time', type='datetime', unit=None, discard=False, canonical_va..."
move_to_front,"[datetime, external_id, author, lat, lon]"
name,value
source_action_id,df5c641b-c50e-4e00-9589-3e13538450c5
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."


id,bbf6a43a-2a91-4b42-81e4-7652ac69afed
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/meteotracker/actions.yaml
match,df5c641b-c50e-4e00-9589-3e13538450c5
name,value
source_action_id,bbf6a43a-2a91-4b42-81e4-7652ac69afed
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data
cache_path,/Users/math/git/IonBeam_bundle/data/cache


id,5c7fa2f3-6fc6-46cf-82c5-2017ec51c0a3
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/meteotracker/actions.yaml
match,bbf6a43a-2a91-4b42-81e4-7652ac69afed
lookup,"{'class': 'const.rd', 'expver': 'const.xxxx', 'stream': 'const.lwda', 'aggregation_type': 'const...."
name,value
source_action_id,5c7fa2f3-6fc6-46cf-82c5-2017ec51c0a3
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data


id,d7b6515a-1de7-4311-b14c-f532692ad1bb
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/meteotracker/actions.yaml
match,5c7fa2f3-6fc6-46cf-82c5-2017ec51c0a3
name,value
source_action_id,d7b6515a-1de7-4311-b14c-f532692ad1bb
state,parsed
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data
cache_path,/Users/math/git/IonBeam_bundle/data/cache


id,11b7904a-12bc-4503-a94e-95932e12c154
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/meteotracker/actions.yaml
match,d7b6515a-1de7-4311-b14c-f532692ad1bb
output,outputs/{source}/odb/{observation_variable}/{observation_variable}_{time_span.start}.odb
name,value
source_action_id,11b7904a-12bc-4503-a94e-95932e12c154
state,odc_encoded
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data


id,bd89b883-fdc6-49de-95b5-86b875debf6c
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,[Match(state = 'odc_encoded')]
config,"{'engine': 'toc', 'spaces': [{'handler': 'Default', 'roots': [{'path': '/Users/math/git/IonBeam_b..."
debug,[]
name,value
source_action_id,bd89b883-fdc6-49de-95b5-86b875debf6c
state,written
mars_id,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."


In [9]:
online = True
if online:
    source = sources[0]
    source.globals.offline = !online
    source.use_cache = True
    incoming_message_stream = source.generate()
    message = next(incoming_message_stream)
    message
else:
    p = Path('/Users/math/git/IonBeam-Deployment/data/inputs/meteotracker/MeteoTracker_62ae154f1d8e11061d4474b2.csv')
    message = FileMessage(
        metadata=MetaData(state='raw',
            source='meteotracker',
            filepath= p,
            variables = list(pd.read_csv(p).columns),             
            ))


In [10]:
from ionbeam.core.bases import FinishMessage, FileMessage, MetaData
from ionbeam.aggregators import TimeAggregator

possible_actions = [a for a in actions if not isinstance(a, Source)]

message_history = []
while True:
    display(message)
    message_history.append(message)
    matching = [action for action in possible_actions if action.matches(message)]
    
    if not matching: 
        print("No more matches")
        break

    print("That matched with: ", [str(a) for a in matching])
    action = matching[0]

    # Special case for the TimeAggregator
    if action.__class__.__name__ == "TimeAggregator":
        list(action.process(message)) #need the list call here to pump the iterator to completion even if it doesn't return anything
        message = next(action.process(FinishMessage("We're done!")))
    else:
        message = next(action.process(message))
        

TabularMessage(metadata=MetaData(source_action_id=UUID('2c4224d7-391f-4d36-a24f-2e33cd365f70'), state='raw', source='meteotracker', observation_variable=None, time_span=TimeSpan(start=Timestamp('2025-01-09 12:41:33+0000', tz='UTC'), end=Timestamp('2025-01-09 12:59:09+0000', tz='UTC')), encoded_format=None, filepath=None, mars_id={}, unstructured={'id': '677fc3fc4b28564e0194af36', 'n_points': 0, 'offset_tz': '+0100', 'start_time': '2025-01-09T12:40:42+00:00', 'author': 'genova_living_lab_28', 'raw_json': {'nPoints': 0, 'nPhotos': 0, 'offsetTZ': '+0100', '_id': '677fc3fc4b28564e0194af36', 'T0': {'avgVal': 0}, 'H': {'avgVal': 0}, 'a': {'avgVal': 0}, 'P': {'avgVal': 0}, 'td': {'avgVal': 0}, 'HDX': {'avgVal': 0}, 'i': {'avgVal': 0}, 'L': {'avgVal': 0}, 'bt': {}, 'tp': {'avgVal': 0}, 'CO2': {}, 'm1': {}, 'm2': {}, 'm4': {}, 'm10': {}, 'n0': {}, 'n1': {}, 'n2': {}, 'n4': {}, 'n10': {}, 'tps': {}, 'EAQ': {}, 'FAQ': {}, 'O3': {}, 'CO': {}, 'SO2': {}, 'AQI': {}, 'Ts': {}, 'sal': {}, 'from': 'Savona', 'startTime': '2025-01-09T12:40:42.000Z', 'by': 'genova_living_lab_28'}, 'end_time': None, 'columns': ['T0', 'H', 'a', 'P', 'td', 'HDX', 'i', 'L', 'tp'], 'living_lab': 'Genoa'}, columns={}, internal_id=None, external_id=None, date=None, time=None), history=[], data=       a                      time    T0   H     P   td    L   HDX     tp  \
0     45 2025-01-09 12:41:33+00:00  10.6  78  1003  7.0  0.0  10.5  283.5   
1     48 2025-01-09 12:41:39+00:00  10.7  75  1003  6.6  0.0  10.4  283.6   
2     53 2025-01-09 12:41:42+00:00  10.7  74  1002  6.3  0.0  10.3  283.7   
3     62 2025-01-09 12:41:48+00:00  10.7  74  1001  6.3  0.0  10.3  283.8   
4     61 2025-01-09 12:41:51+00:00  10.7  72  1001  5.8  0.0  10.1  283.8   
..   ...                       ...   ...  ..   ...  ...  ...   ...    ...   
201  228 2025-01-09 12:58:51+00:00   8.3  54   981 -0.3  NaN   6.0  283.0   
202  228 2025-01-09 12:58:54+00:00   8.3  54   981 -0.3  NaN   6.0  283.0   
203  227 2025-01-09 12:59:00+00:00   8.3  54   981 -0.4  NaN   6.0  283.0   
204  228 2025-01-09 12:59:06+00:00   8.4  53   981 -0.5  NaN   6.0  283.1   
205  228 2025-01-09 12:59:09+00:00   8.4  53   981 -0.4  NaN   6.0  283.1   

       i        lat       lon                        id  
0    NaN  44.304009  8.452655  677fc3fc4b28564e0194af36  
1    NaN  44.304249  8.454907  677fc3fc4b28564e0194af36  
2    NaN  44.304380  8.456047  677fc3fc4b28564e0194af36  
3    NaN  44.304719  8.458304  677fc3fc4b28564e0194af36  
4    0.3  44.304973  8.459402  677fc3fc4b28564e0194af36  
..   ...        ...       ...                       ...  
201  NaN  44.395422  8.494619  677fc3fc4b28564e0194af36  
202  NaN  44.395422  8.494619  677fc3fc4b28564e0194af36  
203  NaN  44.395422  8.494619  677fc3fc4b28564e0194af36  
204  NaN  44.395422  8.494619  677fc3fc4b28564e0194af36  
205  NaN  44.395422  8.494619  677fc3fc4b28564e0194af36  

[412 rows x 13 columns])

That matched with:  ['ExtractMetaData']


name,value
source_action_id,f4c4cf16-a0c2-4b90-b5dd-2e371dcca4f0
state,parsed
source,meteotracker
time_span,2025-01-09T12:41:33+00:00 - 2025-01-09T12:59:09+00:00
mars_id,{}
unstructured,"{'id': '677fc3fc4b28564e0194af36', 'n_points': 0, 'offset_tz': '+0100', 'start_time': '2025-01-09..."
name,value
name,TabularMessage
metadata,"MetaData(source = meteotracker, variable = None)"
name,value


That matched with:  ['CanonicaliseColumns']


TabularMessage(metadata=MetaData(source_action_id=UUID('df5c641b-c50e-4e00-9589-3e13538450c5'), state='parsed', source='meteotracker', observation_variable=None, time_span=TimeSpan(start=Timestamp('2025-01-09 12:41:33+0000', tz='UTC'), end=Timestamp('2025-01-09 12:59:09+0000', tz='UTC')), encoded_format=None, filepath=None, mars_id={}, unstructured={'id': '677fc3fc4b28564e0194af36', 'n_points': 0, 'offset_tz': '+0100', 'start_time': '2025-01-09T12:40:42+00:00', 'author': 'genova_living_lab_28', 'raw_json': {'nPoints': 0, 'nPhotos': 0, 'offsetTZ': '+0100', '_id': '677fc3fc4b28564e0194af36', 'T0': {'avgVal': 0}, 'H': {'avgVal': 0}, 'a': {'avgVal': 0}, 'P': {'avgVal': 0}, 'td': {'avgVal': 0}, 'HDX': {'avgVal': 0}, 'i': {'avgVal': 0}, 'L': {'avgVal': 0}, 'bt': {}, 'tp': {'avgVal': 0}, 'CO2': {}, 'm1': {}, 'm2': {}, 'm4': {}, 'm10': {}, 'n0': {}, 'n1': {}, 'n2': {}, 'n4': {}, 'n10': {}, 'tps': {}, 'EAQ': {}, 'FAQ': {}, 'O3': {}, 'CO': {}, 'SO2': {}, 'AQI': {}, 'Ts': {}, 'sal': {}, 'from': 'Savona', 'startTime': '2025-01-09T12:40:42.000Z', 'by': 'genova_living_lab_28'}, 'end_time': None, 'columns': ['T0', 'H', 'a', 'P', 'td', 'HDX', 'i', 'L', 'tp'], 'living_lab': 'Genoa'}, columns={'datetime': CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was made.'), 'external_id': CanonicalVariable(name='external_id', unit=None, desc='The identifier used by the observation source.'), 'author': CanonicalVariable(name='author', unit=None, desc='The author field from the Meteotracker data'), 'lat': CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'lon': CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'altitude': CanonicalVariable(name='altitude', unit='EPSG:4326', desc='The altitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'air_temperature_near_surface': CanonicalVariable(name='air_temperature_near_surface', unit='K', desc=None), 'relative_humidity_near_surface': CanonicalVariable(name='relative_humidity_near_surface', unit='%', desc=None), 'air_pressure_near_surface': CanonicalVariable(name='air_pressure_near_surface', unit='Pa', desc=None), 'dew_point_temperature': CanonicalVariable(name='dew_point_temperature', unit='K', desc=None), 'solar_radiation_index': CanonicalVariable(name='solar_radiation_index', unit='1', desc=None), 'humidity_index': CanonicalVariable(name='humidity_index', unit='K', desc=None), 'potential_temperature': CanonicalVariable(name='potential_temperature', unit='K', desc=None), 'vertical_temperature_gradient': CanonicalVariable(name='vertical_temperature_gradient', unit='°C/100m', desc=None)}, internal_id=None, external_id=None, date=None, time=None), history=[PreviousActionInfo(action=ActionInfo(name='ExtractMetaData', code=CodeSourceInfo(repo_status='Dirty', git_hash='411ffc13f5b9d36bbfc143d503c88107d5f060f0')), message=MessageInfo(name='TabularMessage', metadata=MetaData(source_action_id=UUID('2c4224d7-391f-4d36-a24f-2e33cd365f70'), state='raw', source='meteotracker', observation_variable=None, time_span=TimeSpan(start=Timestamp('2025-01-09 12:41:33+0000', tz='UTC'), end=Timestamp('2025-01-09 12:59:09+0000', tz='UTC')), encoded_format=None, filepath=None, mars_id={}, unstructured={'id': '677fc3fc4b28564e0194af36', 'n_points': 0, 'offset_tz': '+0100', 'start_time': '2025-01-09T12:40:42+00:00', 'author': 'genova_living_lab_28', 'raw_json': {'nPoints': 0, 'nPhotos': 0, 'offsetTZ': '+0100', '_id': '677fc3fc4b28564e0194af36', 'T0': {'avgVal': 0}, 'H': {'avgVal': 0}, 'a': {'avgVal': 0}, 'P': {'avgVal': 0}, 'td': {'avgVal': 0}, 'HDX': {'avgVal': 0}, 'i': {'avgVal': 0}, 'L': {'avgVal': 0}, 'bt': {}, 'tp': {'avgVal': 0}, 'CO2': {}, 'm1': {}, 'm2': {}, 'm4': {}, 'm10': {}, 'n0': {}, 'n1': {}, 'n2': {}, 'n4': {}, 'n10': {}, 'tps': {}, 'EAQ': {}, 'FAQ': {}, 'O3': {}, 'CO': {}, 'SO2': {}, 'AQI': {}, 'Ts': {}, 'sal': {}, 'from': '

...

That matched with:  ['ComputeMARSIdentifier']


TabularMessage(metadata=MetaData(source_action_id=UUID('5c7fa2f3-6fc6-46cf-82c5-2017ec51c0a3'), state='parsed', source='meteotracker', observation_variable=None, time_span=TimeSpan(start=Timestamp('2025-01-09 12:41:33+0000', tz='UTC'), end=Timestamp('2025-01-09 12:59:09+0000', tz='UTC')), encoded_format=None, filepath=None, mars_id={'class': Key(key='class', value='rd', key_spec=class, reason='Matches', odb_table=None), 'expver': Key(key='expver', value='xxxx', key_spec=expver, reason='Matches', odb_table=None), 'stream': Key(key='stream', value='lwda', key_spec=stream, reason='Matches', odb_table=None), 'aggregation_type': Key(key='aggregation_type', value='tracked', key_spec=aggregation_type, reason='Matches', odb_table=None), 'platform': Key(key='platform', value='meteotracker', key_spec=platform, reason='Matches', odb_table=None), 'date': Key(key='date', value='20250109', key_spec=date, reason='Matches', odb_table=None), 'internal_id': Key(key='internal_id', value=None, key_spec=internal_id, reason='Matches', odb_table=None), 'time': Key(key='time', value=datetime.time(12, 41), key_spec=time, reason='Matches', odb_table=None)}, unstructured={'id': '677fc3fc4b28564e0194af36', 'n_points': 0, 'offset_tz': '+0100', 'start_time': '2025-01-09T12:40:42+00:00', 'author': 'genova_living_lab_28', 'raw_json': {'nPoints': 0, 'nPhotos': 0, 'offsetTZ': '+0100', '_id': '677fc3fc4b28564e0194af36', 'T0': {'avgVal': 0}, 'H': {'avgVal': 0}, 'a': {'avgVal': 0}, 'P': {'avgVal': 0}, 'td': {'avgVal': 0}, 'HDX': {'avgVal': 0}, 'i': {'avgVal': 0}, 'L': {'avgVal': 0}, 'bt': {}, 'tp': {'avgVal': 0}, 'CO2': {}, 'm1': {}, 'm2': {}, 'm4': {}, 'm10': {}, 'n0': {}, 'n1': {}, 'n2': {}, 'n4': {}, 'n10': {}, 'tps': {}, 'EAQ': {}, 'FAQ': {}, 'O3': {}, 'CO': {}, 'SO2': {}, 'AQI': {}, 'Ts': {}, 'sal': {}, 'from': 'Savona', 'startTime': '2025-01-09T12:40:42.000Z', 'by': 'genova_living_lab_28'}, 'end_time': None, 'columns': ['T0', 'H', 'a', 'P', 'td', 'HDX', 'i', 'L', 'tp'], 'living_lab': 'Genoa'}, columns={'datetime': CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was made.'), 'external_id': CanonicalVariable(name='external_id', unit=None, desc='The identifier used by the observation source.'), 'author': CanonicalVariable(name='author', unit=None, desc='The author field from the Meteotracker data'), 'lat': CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'lon': CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'altitude': CanonicalVariable(name='altitude', unit='EPSG:4326', desc='The altitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'air_temperature_near_surface': CanonicalVariable(name='air_temperature_near_surface', unit='K', desc=None), 'relative_humidity_near_surface': CanonicalVariable(name='relative_humidity_near_surface', unit='%', desc=None), 'air_pressure_near_surface': CanonicalVariable(name='air_pressure_near_surface', unit='Pa', desc=None), 'dew_point_temperature': CanonicalVariable(name='dew_point_temperature', unit='K', desc=None), 'solar_radiation_index': CanonicalVariable(name='solar_radiation_index', unit='1', desc=None), 'humidity_index': CanonicalVariable(name='humidity_index', unit='K', desc=None), 'potential_temperature': CanonicalVariable(name='potential_temperature', unit='K', desc=None), 'vertical_temperature_gradient': CanonicalVariable(name='vertical_temperature_gradient', unit='°C/100m', desc=None)}, internal_id=None, external_id=None, date='20250109', time='1241'), history=[PreviousActionInfo(action=ActionInfo(name='ExtractMetaData', code=CodeSourceInfo(repo_status='Dirty', git_hash='411ffc13f5b9d36bbfc143d503c88107d5f060f0')), message=MessageInfo(name='TabularMessage', metadata=MetaData(source_action_id=UUID('2c4224d7-391f-4d36-a24f-2e33cd365f70'), state='raw', source='meteotracker', observation_variable=None, time_span=TimeSpan

That matched with:  ['AddMeteotrackerMetadata']


INFO:ionbeam.sources.meteotracker.metadata:Constructing a new meteotracker sensor object with properties ['datetime', 'external_id', 'author', 'lat', 'lon', 'altitude', 'air_temperature_near_surface', 'relative_humidity_near_surface', 'air_pressure_near_surface', 'dew_point_temperature', 'solar_radiation_index', 'humidity_index', 'potential_temperature', 'vertical_temperature_gradient']


KeyError: 'time'

## Reload and retry failed action

In [ ]:
from ionbeam.core.config_parser import reload_action

print(f"Failed action: {action} Definition: {action.definition_path}")

config, action = reload_action(config, action)

message = next(action.process(message_history[-1]))

message

In [ ]:
m = message_history[-1]
m.metadata.unstructured

In [ ]:
import pyodc as pyodc
from io import BytesIO

f = BytesIO(message.bytes)
df = pyodc.read_odb(f, single = True)
df

In [ ]:
from ionbeam.core.converters import unit_conversions

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100)})

In [ ]:
converter = unit_conversions["°C -> K"]


%timeit df.apply({"A" : converter}, raw = True, by_row = False)

In [ ]:
converter = lambda t: t + 273.15
converter2 = lambda t: t * 1000

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100), "C" : np.arange(100)})

df = df.apply({"A" : converter, "B" : converter2}, raw = True, by_row = False)

df

In [ ]:
converter = lambda 